In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import random
import os

#Convert a Time Series to a Supervised Learning Problem 
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


path = '/Users/jasongangel/Downloads/AIS/data'
filename_read = os.path.join(path, '503000370_clip_NSIDC_EASE.csv')
filename_write = os.path.join(path, 'output.csv')

print(filename_read)
print(filename_write)

dataset = pd.read_csv(filename_read, usecols=['PERIOD', 'LAT', 'LON', 'SPEED_KNOT', 'COG_DEG'], parse_dates=['PERIOD'])

dataset.head()

Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


/Users/jasongangel/Downloads/AIS/data/503000370_clip_NSIDC_EASE.csv
/Users/jasongangel/Downloads/AIS/data/output.csv


,PERIOD,LAT,LON,SPEED_KNOT,COG_DEG
0,2016-01-08 12:00:00,-43.037380,147.27762,0.0,306
1,2016-01-08 13:50:00,-43.037395,147.27760,0.0,191
2,2016-01-08 13:55:00,-43.037388,147.27760,0.0,191
3,2016-01-08 14:20:00,-43.037415,147.27762,0.0,280
4,2016-01-08 14:25:00,-43.037410,147.27762,0.0,280


In [2]:
#CONVERT PERIOD TO UNIX
dataset['UNX'] = dataset['PERIOD'].astype('int64')//10**9

In [3]:
#Transform Coordinates to NSIDC EASE-Grid Global (http://nsidc.org/data/ease)
import shapefile
import pyproj

inProj = pyproj.Proj(init='epsg:4326')
outProj = pyproj.Proj(init='epsg:3410')

def convertCoords(row):
    x2,y2 = pyproj.transform(inProj,outProj,row['LON'],row['LAT'])
   
    return pd.Series({'EASE_LON':x2,'EASE_LAT':y2})

dataset = dataset.join(dataset.apply(convertCoords, axis=1))

In [5]:
#Drop original LAT LON and PERIOD and re-sort Columns
dataset = dataset.drop(['LAT','LON','PERIOD'], axis=1)
dataset = dataset[['UNX', 'EASE_LAT', 'EASE_LON', 'COG_DEG', 'SPEED_KNOT']]

In [6]:
dataset.head()

,UNX,EASE_LAT,EASE_LON,COG_DEG,SPEED_KNOT
0,1452254400,-5.020876e+06,1.418299e+07,306,0.0
1,1452261000,-5.020877e+06,1.418299e+07,191,0.0
2,1452261300,-5.020877e+06,1.418299e+07,191,0.0
3,1452262800,-5.020879e+06,1.418299e+07,280,0.0
4,1452263100,-5.020879e+06,1.418299e+07,280,0.0


# Reframe and Normalize

In [26]:
# specify the number of lag hours
n_posit = 1
n_features = 5

In [27]:
# frame as supervised learning
reframed = series_to_supervised(dataset, n_posit)
reframed.shape

(122094, 10)

In [32]:
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var1(t),var2(t),var3(t),var4(t),var5(t)
1,1.452254e+09,-5020876.0,14182993.0,306.0,0.0,1.452261e+09,-5020877.5,14182992.0,191.0,0.0
2,1.452261e+09,-5020877.5,14182992.0,191.0,0.0,1.452261e+09,-5020876.5,14182992.0,191.0,0.0
3,1.452261e+09,-5020876.5,14182992.0,191.0,0.0,1.452263e+09,-5020879.0,14182993.0,280.0,0.0
4,1.452263e+09,-5020879.0,14182993.0,280.0,0.0,1.452263e+09,-5020878.5,14182993.0,280.0,0.0
5,1.452263e+09,-5020878.5,14182993.0,280.0,0.0,1.452264e+09,-5020880.0,14182994.0,342.0,0.0


In [39]:
#ensure all data is float
reframed = reframed.astype('float32')
type(reframed)

pandas.core.frame.DataFrame

In [42]:
#Standardize Dataframe
import scipy
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
values = scaler.fit_transform(reframed)

# Split and Reshape

In [43]:
# split into train and test sets
n_train_hours = 109886
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

In [44]:
# split into input and outputs
n_obs = n_posit * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)

(109886, 5) 109886 (109886,)


In [45]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_posit, n_features))
test_X = test_X.reshape((test_X.shape[0], n_posit, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(109886, 1, 5) (109886,) (12208, 1, 5) (12208,)


# Design the Network

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

In [ ]:
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

In [ ]:
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -7:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [ ]:
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -7:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

In [ ]:
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)